![k-means by DALL-E3](../data/img/k-means-cover.webp)

# Aprendizaje no supervisado

Se trata de una rama del machine learning cuyo objetivo es agrupar distintos registros de datos en grupos homogeneos en base a sus propias características.

**Ejemplos reales donde se usan algorítmos de aprendizaje no supervisado:**
- Segmentar clientes de un negocio en base a sus características o cómo compran
- Análisis socio-económicos de la población
- Comprimir imágenes agrupando pixeles similares
- Análisis de redes sociales

Ejemplo de un análisis que hizo [@BarriPdmx](https://twitter.com/BarriPdmx/) donde sacó los datos de los tweets que publicaron los fans de una telenovela turca con el hashtag [#babaolmak](https://twitter.com/BarriPdmx/status/1413221249023725569)


![babaolmak por BarriPdmx](../data/img/analisis_rrss_clusters.jpeg)

En el gráfico se ven los distintos perfiles posteando en el hastag y sus interacciones. En base a esas interacciones es capa de agrupar usuarios de una misma comunidad, en este caso del mismo idioma.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

# K Means

El algorítmo k-means es, sorpresa, un algorítmo de aprendizaje no supervisado que agrupa (clusteriza) un conjunto de datos en K grupos. Siendo K un hiperparámetro que elejimos nosotros mismos y que define el número de grupos (clusters) distintos.

Antes de explicar cómo funciona, vamos a ver cómo haríamos nosotros para agrupar datos de forma intuitiva.

## Ejemplo intuitivo

He preguntado a algunos familiares su altura y su peso y los he anotado. Tómate tu tiempo para entender los datos y el código. Después responde a las preguntas en base al gráfico.

In [ ]:
familiar_1 = ["familiar_1", 190, 92]
familiar_2 = ["familiar_2", 173, 58]
familiar_3 = ["familiar_3", 160, 50]
familiar_4 = ["familiar_4", 182, 78]
familiar_5 = ["familiar_5", 122, 23]
familiar_6 = ["familiar_6", 112, 19]

In [ ]:
altura_peso_familiares = pd.DataFrame([familiar_1, familiar_2, familiar_3, familiar_4, familiar_5, familiar_6], columns=['familiar', 'altura', 'peso'])
altura_peso_familiares

In [ ]:
fig = px.scatter(altura_peso_familiares, x='altura', y='peso', title='Altura vs altura de familiares', hover_name='familiar', height=700, width=800)
fig.show()

**Responde a estas preguntas**
- ¿Si tuvieras que agrupar los familiares en dos clusters en base a su altura y peso cúales serían?
- ¿y si tuviertas que agruparlos en tres grupos?

### Respuesta

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42, n_init="auto")
familiares_en_dos_grupos = kmeans.fit_predict(altura_peso_familiares[['altura', 'peso']])
altura_peso_familiares["dos_grupos"] = familiares_en_dos_grupos

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
familiares_en_tres_grupos = kmeans.fit_predict(altura_peso_familiares[['altura', 'peso']])
altura_peso_familiares["tres_grupos"] = familiares_en_tres_grupos

In [ ]:
altura_peso_familiares = altura_peso_familiares.astype({"tres_grupos": "category", "dos_grupos": "category"})
altura_peso_familiares

In [ ]:
fig = px.scatter(altura_peso_familiares, x='altura', y='peso', title='Altura vs altura de familiares', hover_name='familiar', color='dos_grupos', symbol='tres_grupos', height=700, width=800)
fig.show()

Fácil ¿verdad?

## ¿Cómo funciona el algorítmo?

El algoritmo de k-means es relativamente sencillo tanto a nivel conceptual como a nivel matemático. Vamos a ganar una intuición rápida simplemente traduciendo su nombre.
Como ya hemos visto la "K" viene del número de clusters en los que agrupar los datos y "means" quiere decir directamente promedio. De esta forma, lo que hace el algorítmo k-means es:

> *Agrupar el dataset en K clusters, donde cada cluster está caracterizado por su promedio (centroide) con el objetivo de minimizar la distancia de cada dato al centro del grupo al que pertenece*

#### Algorítmo paso a paso

Para lograr generar K grupos cuyo centro esté a la distancia mínima posible de los datos de su grupo, k-means realiza los siguientes pasos:   

1. Elegir el número de clústeres. ESto es un hiperparámetro y generalmente suele basarse en conocimiento del dominio, experimentación o métodos como el método del codo que veremos más adelante.
2. Seleccionar aleatoriamente K puntos de datos como los centroides iniciales. Estos puntos actúan como los centros iniciales de los clústeres.
3. Para cada punto de dato en el conjunto de datos, calcular la distancia (generalmente la distancia euclidiana) desde el punto de dato hasta cada uno de los K centroides.
4. Asignar cada punto de dato al clúster con el centroide más cercano (menor distancia).
5. Después de que todos los puntos sean asignados a clústeres, recalcular los centroides calculando la media de todos los puntos de datos en cada clúster. Esta media se convierte en el nuevo centroide de cada clúster.
6. Repetir los pasos de asignación y actualización (3, 4 y 5) hasta que se cumpla una de las siguientes condiciones:
    - Los centroides no cambian (o el cambio está por debajo de un cierto umbral)
    - La asignación de puntos de datos a clústeres no cambia
    - Se alcanza un número predeterminado de iteraciones
7. Cuando se cumple alguna de las condiciones anteriores consideramos que el algorítmo ha covergido.

**Nota:** K-means es un algorítmo estocástico, por lo para datasets complejos podría no encontrar la solución óptima.

#### Vamos a reproducir el algorítmo a mano

En primer lugar vamos a rescatar el mismo ejemplo de antes.

In [ ]:
# Mismo ejemplo de antes
altura_peso_familiares = altura_peso_familiares.drop(columns=['tres_grupos', 'dos_grupos'])
altura_peso_familiares

Vamos a inventarnos tres centroides (centros de cluster) que no tengan sentido para la relación altura/peso que suele ser normal. Piensa que aunque en este caso la relación altura/peso es algo que conocemos y nos resulta intuitivo, en muchas ocasiones no tendremos una buena intuición sobre los datos.

In [ ]:
# Tres centroides que no tienen sentido para la relación altura/peso
c_1 = [170, 25]
c_2 = [160, 110]
c_3 = [200, 58]
centroides = pd.DataFrame([c_1, c_2, c_3], columns=['altura', 'peso'])
centroides

Ahora vamos a calcular la distancia de cada dato de familiar a los centroides. 

In [ ]:
def distancia_euclidiana(x1, y1, x2, y2):
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

In [ ]:
altura_peso_familiares["distancia_c_1"] = altura_peso_familiares.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], centroides.loc[0, 'altura'], centroides.loc[0, 'peso']), axis="columns")
altura_peso_familiares["distancia_c_2"] = altura_peso_familiares.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], centroides.loc[1, 'altura'], centroides.loc[1, 'peso']), axis="columns")
altura_peso_familiares["distancia_c_3"] = altura_peso_familiares.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], centroides.loc[2, 'altura'], centroides.loc[2, 'peso']), axis="columns")

In [ ]:
altura_peso_familiares["cluster"] = altura_peso_familiares[["distancia_c_1", "distancia_c_2", "distancia_c_3"]].idxmin(axis=1).str[-3:]
altura_peso_familiares

In [ ]:
# Para hacer más fácil el gráfico juntamos los datos con los centroides
centroides["cluster"] = ["c_1", "c_2", "c_3"]
centroides["familiar"] = ["c_1", "c_2", "c_3"]
altura_peso_familiares_centroides = pd.concat([altura_peso_familiares, centroides])
altura_peso_familiares_centroides

In [ ]:
fig = px.scatter(altura_peso_familiares_centroides, x='altura', y='peso', title='Peso vs altura de familiares', hover_name='familiar', color='cluster', height=700, width=800)
fig.show()

Por último recalculamos los nuevos centroides y vemos cómo quedaría ahora en el gráfico.

In [ ]:
centroides_i2 = altura_peso_familiares.groupby('cluster')[["altura", "peso"]].mean().reset_index(drop=True)
centroides_i2["cluster"] = ["c_1", "c_2", "c_3"]
centroides_i2["familiar"] = ["c_1", "c_2", "c_3"]
centroides_i2

In [ ]:
fig = px.scatter(pd.concat([altura_peso_familiares, centroides_i2]), x='altura', y='peso', title='Peso vs altura de familiares', hover_name='familiar', color='cluster', height=700, width=800)
fig.show()

Ahora tocaría volver a iterar, recalcular los puntos más próximos y otra vz el mismo proceso. Para ellos, vamos a empaquetar este proceso un una función y ejecutarlo varias veces.

In [ ]:
def compute_k_means(datos_familia, datos_centroides):
    datos_familia["distancia_c_1"] = datos_familia.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], datos_centroides.loc[0, 'altura'], datos_centroides.loc[0, 'peso']), axis="columns")
    datos_familia["distancia_c_2"] = datos_familia.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], datos_centroides.loc[1, 'altura'], datos_centroides.loc[1, 'peso']), axis="columns")
    datos_familia["distancia_c_3"] = datos_familia.apply(lambda x: distancia_euclidiana(x['altura'], x['peso'], datos_centroides.loc[2, 'altura'], datos_centroides.loc[2, 'peso']), axis="columns")
    datos_familia["cluster"] = datos_familia[["distancia_c_1", "distancia_c_2", "distancia_c_3"]].idxmin(axis=1).str[-3:]
    datos_centroides["cluster"] = ["c_1", "c_2", "c_3"]
    datos_centroides["familiar"] = ["c_1", "c_2", "c_3"]
    nuevos_centroides = datos_familia.groupby('cluster')[["altura", "peso"]].mean().reset_index(drop=True)
    nuevos_centroides["cluster"] = ["c_1", "c_2", "c_3"]
    nuevos_centroides["familiar"] = ["c_1", "c_2", "c_3"]
    return datos_familia, datos_centroides, nuevos_centroides

In [ ]:
altura_peso_familiares, centroides_i2, centroides_i3 = compute_k_means(altura_peso_familiares, centroides_i2)

In [ ]:
fig = px.scatter(pd.concat([altura_peso_familiares, centroides_i3]), x='altura', y='peso', title='Peso vs altura de familiares', hover_name='familiar', color='cluster', height=700, width=800)
fig.show()

In [ ]:
altura_peso_familiares, centroides_i3, centroides_i4 = compute_k_means(altura_peso_familiares, centroides_i3)

In [ ]:
fig = px.scatter(pd.concat([altura_peso_familiares, centroides_i4]), x='altura', y='peso', title='Peso vs altura de familiares', hover_name='familiar', color='cluster', height=700, width=800)
fig.show()

In [ ]:
altura_peso_familiares, centroides_i4, centroides_i5 = compute_k_means(altura_peso_familiares, centroides_i3)

In [ ]:
fig = px.scatter(pd.concat([altura_peso_familiares, centroides_i5]), x='altura', y='peso', title='Peso vs altura de familiares', hover_name='familiar', color='cluster', height=700, width=800)
fig.show()

Recordamos las condiciones cumplidas:
    - Los centroides no cambian (o el cambio está por debajo de un cierto umbral)
    - La asignación de puntos de datos a clústeres no cambia
    - Se alcanza un número predeterminado de iteraciones

No hemos establecido previamente un umbral para la primera condición, pero a simple vista es evidente que se cumple. Además, también se cumple la segunda condición.
Podemos decir que el algoritmo ha convergido.

## K-means en scikit-learn

In [ ]:
altura_peso_familiares = pd.DataFrame([familiar_1, familiar_2, familiar_3, familiar_4, familiar_5, familiar_6], columns=['familiar', 'altura', 'peso'])
altura_peso_familiares

In [ ]:
k_grupos = 3
init = [[170, 25], [160, 110], [200, 58]]
# init='k-means++'
kmeans = KMeans(n_clusters=k_grupos, init=init, n_init="auto", random_state=42)

familiares_en_k_grupos = kmeans.fit_predict(altura_peso_familiares[['altura', 'peso']])
altura_peso_familiares["cluster"] = familiares_en_k_grupos
altura_peso_familiares = altura_peso_familiares.astype({"cluster": "category"})

In [ ]:
altura_peso_familiares

### Centroides
Extraemos los centroides directamente del objeto k-means de scikit-learn.   

Los centroides no vienen ordenados con el mismo orden que los clusters de los datos, por eso les ponemos un numero distinto.

In [ ]:
centroides = kmeans.cluster_centers_
datos_centroides = pd.DataFrame(centroides, columns=["altura", "peso"])
datos_centroides["cluster"] = ["4", "4", "4"]
datos_centroides["familiar"] = ["4", "4", "4"]
datos_centroides

### Inercia
Suma de los cuadrados de las distancias de cada dato a su centroide correspondiente. Se puede interpretar como una métrica de calidad. Cuanto más pequeña es la inercia, más cerca están los puntos de su centroide.


In [ ]:
distnacia_a_centroides = kmeans.inertia_
distnacia_a_centroides

### Visualización

In [ ]:
fig = px.scatter(pd.concat([altura_peso_familiares, datos_centroides]), x='altura', y='peso', title='Altura vs altura de familiares', hover_name='familiar', color="cluster", height=700, width=800)
fig.show()

## ¿Cuántos clusters (K) elegir?
Cuando no sabemos a priori cuál es el número óptimo o no tenemos intuición sobre los datos.

## Método del codo (elbow method)

Se trata de un método empirico muy sencillo:
1. Se calcula k-means para distintos valores de K
2. Se obtienen las inercias de cada iteración y se visualizan
3. Cuando detectemos que la gráfica de la inercia se "aplana", detectamos el "codo"
4. Elegirimos el K correspondiente al codo como el óptimo

En esta ocasión, aunque seguimos con la misma intuición de la relación altura/peso, vamos a generar un dataset mucho más grande el cuál nos despistaría si no lo conocieramos de antemano.

In [ ]:
# Seed para reproducibilidad
np.random.seed(42)

# g_1: Promedio altura ~ 165 cm, peso ~ 60 kg
altura_g1 = np.random.normal(165, 10, 100)
peso_g1 = np.random.normal(60, 8, 100)

# g_2: Promedio altura ~ 180 cm, peso ~ 80 kg
altura_g2 = np.random.normal(180, 10, 100)
peso_g2 = np.random.normal(80, 12, 100)

# g_3: Promedio altura ~ 120 cm, peso ~ 30 kg
altura_g3 = np.random.normal(120, 15, 100)
peso_g3 = np.random.normal(30, 10, 100)

alturas = np.concatenate([altura_g1, altura_g2, altura_g3])
pesos = np.concatenate([peso_g1, peso_g2, peso_g3])
grupos = ['g_1']*100 + ['g_2']*100 + ['g_3']*100

# Creating a DataFrame
df = pd.DataFrame({
    'altura': alturas,
    'peso': pesos,
    'grupo': grupos
})

df.head()

Vamos a comprobar qué aspecto tienen nuestros datos.

In [ ]:
fig = px.scatter(df, x='altura', y='peso', title='Altura vs altura de familiares', hover_name='grupo', color="grupo", height=700, width=800)
fig.show()

Ahora vamos a crear una función que calcule la inercia de cada K-means desde K = 1 hasta k = 10 (por ejemplo).

In [ ]:
import plotly.graph_objects as go
def elbow_method(data, max_clusters=10):
    inercias = []

    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
        kmeans.fit(data)
        inercias.append(kmeans.inertia_)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, 11)), y=inercias, mode='lines+markers', name='inercias'))
    fig.update_layout(title='Método del codo para elegir el K optimo',
                    xaxis_title='Número de clusters',
                    yaxis_title='Inercias',
                    xaxis=dict(tickmode='linear', tick0=1, dtick=1))
    fig.show()

In [ ]:
elbow_method(df[['altura', 'peso']], max_clusters=10)

En este caso, aún sin saber que los datos originales se agrupan en 3 clusters, el método del codo nos daría un K optimo de 3.

P.d. Podría ser una buena duda si el K optimo es 2, 3 o 4, pero eso lo dejo para que lo pienses tú :D